In [67]:
import pandas as pd
file_name = 'Chiayi_tripadvisor_top500'
df = pd.read_csv('../0815/'+file_name+'.csv')

In [22]:
# 計算重複數量
count = 0
for i, value in df['uri'].value_counts().iteritems():
    if value > 1:
        count +=1
print(count)

84


### 顯示重複

In [3]:
df.loc[df['uri'] == df['uri'].value_counts().index[0]]

,title,paiming,uri,lat,lng,recommend_price,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,offical_img_uri,facility,room,agoda_price
284,砝傣岸民宿,"花蓮排名第 285 的超值住宿地點，共 2,231 家",https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.659290,121.419960,"NT$1,212",光復鄉大進村中興路176巷1弄2號,3.5,花蓮,光復鄉,-NT$1,4.0,1.0,https://media-cdn.tripadvisor.com/media/partne...,NaN,NaN,"NT$2,174"
305,砝傣岸民宿,"花蓮排名第 306 的超值住宿地點，共 2,231 家",https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.328011,121.321114,"NT$1,212",光復鄉大進村中興路176巷1弄2號,3.5,花蓮,光復鄉,-NT$1,4.0,1.0,https://static.tacdn.com/img2/branding/rebrand...,NaN,NaN,NaN
346,砝傣岸民宿,"花蓮排名第 347 的超值住宿地點，共 2,231 家",https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.972549,121.614670,"NT$1,212",光復鄉大進村中興路176巷1弄2號,3.5,花蓮,光復鄉,-NT$1,4.0,1.0,https://media-cdn.tripadvisor.com/media/partne...,NaN,NaN,NaN
366,砝傣岸民宿,"花蓮排名第 367 的超值住宿地點，共 2,231 家",https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.181171,121.494217,"NT$1,212",光復鄉大進村中興路176巷1弄2號,5.0,花蓮,光復鄉,-NT$1,4.0,1.0,https://media-cdn.tripadvisor.com/media/partne...,NaN,NaN,"NT$9,046"


### 只保留相同飯店名稱/ uri的第一筆

In [68]:
rows_list = []
# 把row存到list, 之後再轉df
for i, value in df['uri'].value_counts().iteritems():
    # 轉成dict再存到 list
    rows_list.append(df.loc[df['uri'] == i].head(1).to_dict('records')[0])
    
new_df = pd.DataFrame(rows_list)

確定沒有重複的飯店

In [60]:
new_df['uri'].value_counts()[new_df['uri'].value_counts() > 1].index

Index([], dtype='object')

---
#### 移除用不到的特徵

In [69]:
new_df.drop(['paiming', 'offical_img_uri'], axis=1, inplace=True)

空的rating跟comment補0, 表示沒有人評價過

In [70]:
new_df['avg_rating'].fillna(0, inplace=True)
new_df['comment_count'].fillna(0, inplace=True)

---
#### drop掉空值過多的欄位

In [71]:
new_df.isna().sum()/ len(df)

title              0.000000
uri                0.000000
lat                0.191919
lng                0.191919
recommend_price    0.181818
hotel_address      0.000000
hotel_star         0.090909
hotel_city         0.000000
hotel_section      0.000000
price_range        0.131313
avg_rating         0.000000
comment_count      0.000000
facility           0.383838
room               0.363636
agoda_price        0.848485
dtype: float64

只有10筆是完整沒有空值

In [9]:
new_df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 5 to 468
Data columns (total 15 columns):
title              41 non-null object
uri                41 non-null object
lat                41 non-null float64
lng                41 non-null float64
recommend_price    41 non-null object
hotel_address      41 non-null object
hotel_star         41 non-null float64
hotel_city         41 non-null object
hotel_section      41 non-null object
price_range        41 non-null object
avg_rating         41 non-null float64
comment_count      41 non-null float64
facility           41 non-null object
room               41 non-null object
agoda_price        41 non-null object
dtypes: float64(5), object(10)
memory usage: 5.1+ KB


In [10]:
new_df.dropna(axis=1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 7 columns):
title            477 non-null object
uri              477 non-null object
hotel_address    477 non-null object
hotel_city       477 non-null object
hotel_section    477 non-null object
avg_rating       477 non-null float64
comment_count    477 non-null float64
dtypes: float64(2), object(5)
memory usage: 26.2+ KB


暫時剔除以下的feature

In [63]:
new_df.drop(['recommend_price', 'facility','room', 'agoda_price'], axis=1, inplace=True)

再踢出row有空值的

In [64]:
new_df.dropna(inplace=True)
print(new_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 1 to 97
Data columns (total 11 columns):
title            71 non-null object
uri              71 non-null object
lat              71 non-null float64
lng              71 non-null float64
hotel_address    71 non-null object
hotel_star       71 non-null float64
hotel_city       71 non-null object
hotel_section    71 non-null object
price_range      71 non-null object
avg_rating       71 non-null float64
comment_count    71 non-null float64
dtypes: float64(5), object(6)
memory usage: 6.7+ KB
None


### 存成新的表

In [65]:
new_df.to_csv('./'+file_name+'_clean.csv', index=False, encoding='utf_8_sig')